## Random Forrest & Decision Forrest

In [1]:
## imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [56]:
## dictionary for the dataset name & paths 
DATASETS ={
    "small":"../Data/small/glass.csv",
    "medium":"../Data/medium/drug_consumption.data",
    "large":"../Data/large/c2k_data_comma.csv"
               }

## Data Split Parameters 
TRAIN_SIZE = 0.8
TEST_SIZE = 1 - TRAIN_SIZE

## Random Forest Parameters
NUM_TREES = 100 ## 1,10,25,50,75,100 
NUM_FEATURES = 10  ## 1,3, int(log_2(M)+1), sqrt(M), where M is the number of features


In [57]:
## reading the small dataset 
small = pd.read_csv(DATASETS["small"],header=None)
## checking empty values 
print(small.isna().sum().sum() == small.isnull().sum().sum(),"No empty values")

True No empty values


In [64]:
## data splitting 
from sklearn.model_selection import train_test_split
## split into the train and test sets, stratify the data
train, test = train_test_split(small,test_size=TEST_SIZE)